In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import pandas as pd
import os

metadata = pd.read_csv('C:/Users/thott/Downloads/Voiceset/en/validated.tsv', sep="\t")

print(metadata.head())

print(metadata.columns)

                                           client_id  \
0  031903093b6fa1aeb0a243843eb9ed57baf6e99d1f8f92...   
1  058fe5b1170aa09ef3f1092b179384639bc46ac53c1675...   
2  08190396a5c298331813531d1a832b56d8ffe44aaedcb7...   
3  14698ee63cabe08b43f0faa93304202d1e6ffeaa2cdf86...   
4  28d8f8a88afad9eb9e5b36ee84bd4c5ba84137310da15f...   

                           path  \
0  common_voice_en_41383256.mp3   
1  common_voice_en_41823983.mp3   
2  common_voice_en_41881685.mp3   
3  common_voice_en_41799514.mp3   
4  common_voice_en_41552032.mp3   

                                         sentence_id  \
0  f19a785911b1a3b1338e3eb5cc785e58b8381d21ec7c33...   
1  f50360e1be367d8155b3c8340f0b3d38d1e6701df79dc5...   
2  f4f3a5714cc36a9abbabf78a33feb4a9c368005f1f4bf5...   
3  f4d04f6e48777c3ad180c629858a19fdfa4cb875d2bb22...   
4  f262ed293fa5fe0986d1e7a80b5bbae11205f8089a1857...   

                                            sentence  sentence_domain  \
0  The outer rim has undergone some erosio

In [2]:
from pydub import AudioSegment
import os

input_directory = "C:/Users/thott/Downloads/Voiceset/en/clips"
output_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_wav"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def convert_mp3_to_wav(mp3_path, wav_path):
    try:
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format="wav")
        print(f"Successfully converted: {mp3_path} to {wav_path}")
    except Exception as e:
        print(f"Error converting {mp3_path}: {e}")

for filename in os.listdir(input_directory):
    if filename.endswith(".mp3"):
        mp3_path = os.path.join(input_directory, filename).replace("\\", "/")
        wav_filename = filename.replace(".mp3", ".wav")
        wav_path = os.path.join(output_directory, wav_filename).replace("\\", "/")
        
        convert_mp3_to_wav(mp3_path, wav_path)

In [3]:
import librosa
import soundfile as sf
import os

def resample_audio(audio_file, target_sample_rate=16000, output_directory="C:/Users/thott/Downloads/Voiceset/en/clips_resampled"):
    try:
        y, sr = librosa.load(audio_file, sr=None)
        
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sample_rate)
        
        os.makedirs(output_directory, exist_ok=True)
        
        output_file = os.path.join(output_directory, os.path.basename(audio_file).replace(".wav", "_resampled.wav"))
        sf.write(output_file, y_resampled, target_sample_rate)
        print(f"Successfully resampled {audio_file} to {output_file}.")
        
    except Exception as e:
        print(f"Error resampling {audio_file}: {e}")

audio_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_wav"
resampled_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_resampled"

for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        audio_file = os.path.join(audio_directory, filename)
        resample_audio(audio_file, output_directory=resampled_directory)

In [4]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import os

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

def transcribe_audio(audio_path):
    try:
        audio_input, sample_rate = torchaudio.load(audio_path)
        if audio_input.shape[0] > 1:
            audio_input = torch.mean(audio_input, dim=0, keepdim=True)
        if sample_rate != 16000:
            transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            audio_input = transform(audio_input)
        audio_input = audio_input.squeeze(0)  # (num_samples,)
        inputs = processor(audio_input, return_tensors="pt", sampling_rate=16000)
        with torch.no_grad():
            logits = model(inputs.input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]
        
        return transcription

    except Exception as e:
        print(f"Error transcribing {audio_path}: {e}")
        return None
audio_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_resampled"
transcriptions = {}
for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        audio_path = os.path.join(audio_directory, filename)
        transcription = transcribe_audio(audio_path)
        if transcription:
            transcriptions[filename] = transcription
            print(f"Transcription for {filename}: {transcription}")

In [5]:
transcriptions = {}

audio_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_resampled"

for index, row in metadata.iterrows():
    filename = os.path.join(audio_directory, row['path'])
    ground_truth = row['sentence']

    if filename in transcriptions:
        hypothesis = transcriptions[filename]
        accent_group = row['accents']

In [6]:
import jiwer
import pandas as pd

metadata = pd.read_csv('C:/Users/thott/Downloads/Voiceset/en/validated.tsv', sep="\t")

transformer = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemovePunctuation(),
    jiwer.Strip(),
    jiwer.RemoveMultipleSpaces()
])

def calculate_wer(ground_truth, hypothesis):
    return jiwer.wer(ground_truth, hypothesis, truth_transform=transformer, hypothesis_transform=transformer)

accent_groups_wer = {}

for index, row in metadata.iterrows():
    audio_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_resampled"
    filename = os.path.join(audio_directory, row['path'])
    ground_truth = row['sentence']
    
    if filename in transcriptions:
        hypothesis = transcriptions[filename]
        accent_group = row['accent']
        wer = calculate_wer(ground_truth, hypothesis)
        
        if accent_group not in accent_groups_wer:
            accent_groups_wer[accent_group] = []
        
        accent_groups_wer[accent_group].append(wer)

for accent_group, wer_list in accent_groups_wer.items():
    average_wer = sum(wer_list) / len(wer_list)
    print(f"Average WER for {accent_group}: {average_wer}")

In [7]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
import numpy as np

y_true = np.array(metadata['sentence'])
y_pred = np.array([transcriptions.get(filename, '') for filename in metadata['path']])

metadata['accents'] = metadata['accents'].fillna("unknown")
metadata['accents'] = metadata['accents'].astype(str)

group = np.array(metadata['accents'])
metric_frame = MetricFrame(metrics=accuracy_score, y_true=y_true, y_pred=y_pred, sensitive_features=group)

print("Accuracy per accent group:")
print(metric_frame.by_group)

Accuracy per accent group:
sensitive_feature_0
British with slightly Brazilian accent,British and slightly Brazilian      0.0
Californian Accent                                                         0.0
Canadian English                                                           0.0
Canadian English,English Hungarian                                         0.0
England English                                                            0.0
England English,Kent                                                       0.0
Filipino                                                                   0.0
French speaking english                                                    0.0
India and South Asia (India, Pakistan, Sri Lanka)                          0.0
India and South Asia (India, Pakistan, Sri Lanka),South India,Kannadiga    0.0
Israeli English                                                            0.0
Japan English                                                              0.0
Japan

In [8]:
from pydub import AudioSegment
import random

def pitch_shift_audio(audio_path, n_steps=2, output_directory="C:/Users/thott/Downloads/Voiceset/en/clips_augmented"):
    try:
        audio = AudioSegment.from_wav(audio_path)
        shifted_audio = audio._spawn(audio.raw_data)
        shifted_audio = shifted_audio.set_frame_rate(int(audio.frame_rate * random.choice([0.8, 1.2])))
        
        audio_name = os.path.basename(audio_path).replace(".wav", "_augmented.wav")
        output_path = os.path.join(output_directory, audio_name)
        shifted_audio.export(output_path, format="wav")
        print(f"Successfully augmented and saved: {output_path}")
        
    except Exception as e:
        print(f"Error augmenting {audio_path}: {e}")

audio_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_wav"
augmented_directory = "C:/Users/thott/Downloads/Voiceset/en/clips_augmented"

for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        audio_path = os.path.join(audio_directory, filename)
        pitch_shift_audio(audio_path, output_directory=augmented_directory)

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41236242_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247165_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247167_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247169_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247171_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247172_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247197_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41247199_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267073_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267075_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267078_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267143_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267148_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267150_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267151_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41267153_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41278585_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281272_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281273_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281274_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281275_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281276_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281277_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41281278_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282674_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282675_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282676_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282677_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282678_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282679_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282680_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41282681_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289038_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289039_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289040_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289041_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289067_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289068_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289069_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41289070_augmen

...

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907468_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907469_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907470_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907471_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907472_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907477_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907478_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41907479_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909015_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909016_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909017_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909018_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909019_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909021_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909025_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909027_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909177_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909183_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909184_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909185_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909186_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909187_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909188_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909189_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909334_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909335_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909336_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909337_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909338_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909339_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909340_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41909341_augmen

Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910182_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910183_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910184_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910185_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910191_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910192_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910193_augmented.wav
Successfully augmented and saved: C:/Users/thott/Downloads/Voiceset/en/clips_augmented\common_voice_en_41910194_augmen

In [9]:
import os
import random
import tkinter as tk
from tkinter import filedialog, messagebox
from pydub import AudioSegment

def select_input_folder():
    folder = filedialog.askdirectory()
    if folder:
        input_folder.set(folder)
        select_random_file()

def select_output_folder():
    folder = filedialog.askdirectory()
    if folder:
        output_folder.set(folder)

def select_random_file():
    folder = input_folder.get()
    if not folder:
        messagebox.showerror("Error", "Please select an input folder first.")
        return
    
    files = [f for f in os.listdir(folder) if f.endswith(".wav")]
    if not files:
        messagebox.showerror("Error", "No .wav files found in the selected folder.")
        return
    
    selected_file.set(random.choice(files))
    file_label.config(text=f"Selected File: {selected_file.get()}")

def process_audio():
    if not input_folder.get() or not output_folder.get():
        messagebox.showerror("Error", "Please select both input and output folders.")
        return

    if not selected_file.get():
        messagebox.showerror("Error", "No file selected.")
        return

    input_path = os.path.join(input_folder.get(), selected_file.get())
    output_path = os.path.join(output_folder.get(), selected_file.get().replace(".wav", "_augmented.wav"))

    try:
        audio = AudioSegment.from_wav(input_path)
        shifted_audio = audio._spawn(audio.raw_data)
        shifted_audio = shifted_audio.set_frame_rate(int(audio.frame_rate * random.choice([0.8, 1.2])))
        shifted_audio.export(output_path, format="wav")

        messagebox.showinfo("Success", f"Processed file saved:\n{output_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Error processing file: {e}")

root = tk.Tk()
root.title("Audio Augmentation")
root.geometry("400x250")

input_folder = tk.StringVar()
output_folder = tk.StringVar()
selected_file = tk.StringVar()

tk.Label(root, text="Input Folder:").pack()
tk.Button(root, text="Select Folder", command=select_input_folder).pack()

file_label = tk.Label(root, text="No file selected", fg="blue")
file_label.pack()

tk.Label(root, text="Output Folder:").pack()
tk.Button(root, text="Select Folder", command=select_output_folder).pack()

tk.Button(root, text="Process Random File", command=process_audio, bg="green", fg="white").pack(pady=10)

root.mainloop()